In [ ]:
import qsharp
import os, sys

notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from util import plot

In [ ]:
%%qsharp

operation Independent(measurement_basis : Pauli) : Result[] {
    use (one, two) = (Qubit(), Qubit());

    let result_one = Measure([measurement_basis], [one]);
    let result_two = Measure([measurement_basis], [two]);
    ResetAll([one, two]);

    [result_one, result_two]
}

operation Entangled(measurement_basis : Pauli) : Result[] {
    use (one, two) = (Qubit(), Qubit());

    // create entangled state
    H(one);
    CNOT(one, two);

    let result_one = Measure([measurement_basis], [one]);
    let result_two = Measure([measurement_basis], [two]);
    ResetAll([one, two]);

    [result_one, result_two]
}

In [ ]:
results = qsharp.run("Independent(PauliZ)", shots=1000)
plot(results, title="Independent electron spin measurement - Z basis")

In [ ]:
results = qsharp.run("Independent(PauliX)", shots=1000)
plot(results, title="Independent electron spin measurement - X basis")

In [ ]:
results = qsharp.run("Entangled(PauliZ)", shots=1000)
plot(results, title="Entangled electrons spin measurement - Z basis")

In [ ]:
results = qsharp.run("Entangled(PauliX)", shots=1000)
plot(results, title="Entangled electrons spin measurement - X basis")